In [8]:
from agno.agent import Agent
from agno.models.openai import OpenAILike
from agno.vectordb.lancedb import LanceDb
from agno.vectordb.search import SearchType
from agno.embedder.openai import OpenAIEmbedder
from agno.tools.reasoning import ReasoningTools
from agno.knowledge import AgentKnowledge
from agno.playground import Playground, serve_playground_app

#------------------ settings ------------------
import os
from dotenv import load_dotenv
load_dotenv()
api_key=os.getenv("QWEN_API_KEY")
base_url=os.getenv("QWEN_API_BASE_URL")
local_base_url = 'http://192.168.0.166:8000/v1'
local_model_name = 'Qwen3-235B'
model_name = 'qwen-plus-latest'
embedding_model_id = 'text-embedding-v4'

local_settings = {
  'api_key' : '123',
  'base_url' : local_base_url,
  'id' : local_model_name
}

qwen_settings = {
  'api_key' : api_key,
  'base_url' : base_url,
  'id' : model_name
}

deepseek_settings = {
  'api_key' : os.getenv("DEEPSEEK_API_KEY"),
  'base_url' : os.getenv("DEEPSEEK_API_BASE_URL"),
  'id' : 'deepseek-chat'
}

settings = deepseek_settings
#------------------ settings ------------------

vector_db = LanceDb(
    table_name="contact_table",
    uri="D:\\projects\\LLM-Explore\\src\\agent\\contact-query\\tmp\\contact_vectors.lancedb",
    search_type=SearchType.hybrid,
    embedder=OpenAIEmbedder(id=embedding_model_id,api_key=api_key,base_url=base_url, dimensions=2048),
)


knowledge_base = AgentKnowledge(vector_db=vector_db)

# add_references=True 就是传统rag的做法，会将找到的相关文档都放在用户的context中，而现代的做法会通过search_knowledge_base进行工具查询然后得到相关结果，建议先不使用add_references=True，如果性能有问题再进行相关测试进行评估

agent = Agent(
    model=OpenAILike(**settings),
    name='Contact_Query_Agent',
    instructions=['查询合同详情的时候请列出所有数据，严禁遗漏任何条目','禁止虚构和假设任何数据','如果需要进行合同比对的时候，请按需**分别**查出所有项目后再进行比对','必须使用简体中文回复'],
    knowledge=knowledge_base,
    add_history_to_messages=True,
    num_history_responses=20,
    tools=[ReasoningTools(add_instructions=True)],
    markdown=True,
    # add_references=True,
    stream=True,
    stream_intermediate_steps=True,
    telemetry=False,
    debug_mode=False,
)

In [2]:
response = agent.run(message='包头钢铁烧结机余热项目和宝钢德盛烧结余热项目的余热锅炉价格相差多少')

INFO Found 5 documents

INFO Found 5 documents

In [4]:
response.messages

[Message(role='system', content='<instructions>\n- 查询合同详情的时候请列出所有数据，严禁遗漏任何条目\n- 禁止虚构和假设任何数据\n- 如果需要进行合同比对的时候，请按需**分别**查出所有项目后再进行比对\n- 必须使用简体中文回复\n</instructions>\n\n<additional_information>\n- Use markdown to format your answers.\n</additional_information>', name=None, tool_call_id=None, tool_calls=None, audio=None, images=None, videos=None, files=None, audio_output=None, image_output=None, thinking=None, redacted_thinking=None, provider_data=None, citations=None, reasoning_content=None, tool_name=None, tool_args=None, tool_call_error=None, stop_after_tool_call=False, add_to_agent_memory=True, from_history=False, metrics=MessageMetrics(input_tokens=0, output_tokens=0, total_tokens=0, audio_tokens=0, input_audio_tokens=0, output_audio_tokens=0, cached_tokens=0, cache_write_tokens=0, reasoning_tokens=0, prompt_tokens=0, completion_tokens=0, prompt_tokens_details=None, completion_tokens_details=None, additional_metrics=None, time=None, time_to_first_token=None, timer=None), references=Non

In [9]:
from agno.run.response import RunEvent

response_iter = agent.run(message='包头钢铁烧结机余热项目和宝钢德盛烧结余热项目的余热锅炉价格相差多少',stream=True,stream_intermediate_steps=True)
for single_response in response_iter:
    if single_response.event == RunEvent.run_started:
        print('run started')
    elif single_response.event == RunEvent.run_response:
        print(single_response.content,end='',flush=True)
    else:
        print(f'event: {single_response.event}', single_response)

run started
event: ToolCallStarted RunResponse(content='', content_type='str', thinking=None, reasoning_content=None, event='ToolCallStarted', messages=None, metrics=None, model='deepseek-chat', model_provider=None, run_id='48d9c5a1-aeb3-4cb9-8ae3-de10c210e29f', agent_id='5fc97344-63cf-4888-9380-275d62d74480', session_id='94884415-0da6-458b-b565-1cf66269bf6b', workflow_id=None, tools=[ToolExecution(tool_call_id='call_0_b1870115-41af-452e-9b99-9de7aad6f255', tool_name='think', tool_args={'title': '明确查询目标', 'thought': '用户希望了解包头钢铁烧结机余热项目和宝钢德盛烧结余热项目的余热锅炉价格差异。需要分别查询这两个项目的余热锅炉价格，然后计算差值。', 'action': '分别查询两个项目的余热锅炉价格', 'confidence': 0.95}, tool_call_error=None, result=None, metrics=None, stop_after_tool_call=False, created_at=1749352209, requires_confirmation=None, confirmed=None, confirmation_note=None, requires_user_input=None, user_input_schema=None, external_execution_required=None)], formatted_tool_calls=None, images=None, videos=None, audio=None, response_audio=None, citations=None, extr

INFO Found 5 documents

event: ToolCallCompleted RunResponse(content='', content_type='str', thinking=None, reasoning_content='## 明确查询目标\n用户希望了解包头钢铁烧结机余热项目和宝钢德盛烧结余热项目的余热锅炉价格差异。需要分别查询这两个项目的余热锅炉价格，然后计算差值。\nAction: 分别查询两个项目的余热锅炉价格\nConfidence: 0.95\n\n', event='ToolCallCompleted', messages=None, metrics=None, model='deepseek-chat', model_provider=None, run_id='48d9c5a1-aeb3-4cb9-8ae3-de10c210e29f', agent_id='5fc97344-63cf-4888-9380-275d62d74480', session_id='94884415-0da6-458b-b565-1cf66269bf6b', workflow_id=None, tools=[ToolExecution(tool_call_id='call_0_b1870115-41af-452e-9b99-9de7aad6f255', tool_name='think', tool_args={'title': '明确查询目标', 'thought': '用户希望了解包头钢铁烧结机余热项目和宝钢德盛烧结余热项目的余热锅炉价格差异。需要分别查询这两个项目的余热锅炉价格，然后计算差值。', 'action': '分别查询两个项目的余热锅炉价格', 'confidence': 0.95}, tool_call_error=False, result='Step 1:\nTitle: 明确查询目标\nReasoning: 用户希望了解包头钢铁烧结机余热项目和宝钢德盛烧结余热项目的余热锅炉价格差异。需要分别查询这两个项目的余热锅炉价格，然后计算差值。\nAction: 分别查询两个项目的余热锅炉价格\nConfidence: 0.95', metrics=MessageMetrics(input_tokens=0, output_tokens=0, total_tokens=0, 

INFO Found 5 documents

event: ToolCallCompleted RunResponse(content='', content_type='str', thinking=None, reasoning_content='## 明确查询目标\n用户希望了解包头钢铁烧结机余热项目和宝钢德盛烧结余热项目的余热锅炉价格差异。需要分别查询这两个项目的余热锅炉价格，然后计算差值。\nAction: 分别查询两个项目的余热锅炉价格\nConfidence: 0.95\n\n', event='ToolCallCompleted', messages=None, metrics=None, model='deepseek-chat', model_provider=None, run_id='48d9c5a1-aeb3-4cb9-8ae3-de10c210e29f', agent_id='5fc97344-63cf-4888-9380-275d62d74480', session_id='94884415-0da6-458b-b565-1cf66269bf6b', workflow_id=None, tools=[ToolExecution(tool_call_id='call_0_b1870115-41af-452e-9b99-9de7aad6f255', tool_name='think', tool_args={'title': '明确查询目标', 'thought': '用户希望了解包头钢铁烧结机余热项目和宝钢德盛烧结余热项目的余热锅炉价格差异。需要分别查询这两个项目的余热锅炉价格，然后计算差值。', 'action': '分别查询两个项目的余热锅炉价格', 'confidence': 0.95}, tool_call_error=False, result='Step 1:\nTitle: 明确查询目标\nReasoning: 用户希望了解包头钢铁烧结机余热项目和宝钢德盛烧结余热项目的余热锅炉价格差异。需要分别查询这两个项目的余热锅炉价格，然后计算差值。\nAction: 分别查询两个项目的余热锅炉价格\nConfidence: 0.95', metrics=MessageMetrics(input_tokens=0, output_tokens=0, total_tokens=0, 